In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/OK_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1008918710,OK,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14
1,1994657208,OK,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,4.010911e+14
2,2427157100,OK,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14
3,6097937201,OK,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14
4,6262678403,OK,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,4.013995e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1008918710,OK,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14
1,1994657208,OK,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,4.010911e+14
2,2427157100,OK,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14
3,6097937201,OK,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14
4,6262678403,OK,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,4.013995e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
64,4920347210,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
423,6223537005,OK,"7001 S Hwy 69A, MIAMI, OK",35.078999,-106.715022,NaN
491,3319208509,OK,"7600 S Highway 69A, Miami, OK",37.196334,-94.191716,NaN
573,8230827006,OK,"7600 S Hwy 69 A, MIAMI, OK",37.196334,-94.191716,NaN
644,4993557204,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
1009,3053198401,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
1010,9592567002,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
2210,2917617110,OK,"1715 South Highway 183, Clinton, OK",30.220229,-97.683076,NaN
2393,8825047105,OK,"809 NE HWY 60 809 NE HWY 60, SEILING, OK",41.285247,-99.385728,NaN
2549,5893907210,OK,"1003 HWY 64 NORTH, BUFFALO, OK",35.922512,-75.678195,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('40')] #OK
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/26 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

26


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11621/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
64,4920347210,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,[]
423,6223537005,OK,"7001 S Hwy 69A, MIAMI, OK",35.078999,-106.715022,[]
491,3319208509,OK,"7600 S Highway 69A, Miami, OK",37.196334,-94.191716,[]
573,8230827006,OK,"7600 S Hwy 69 A, MIAMI, OK",37.196334,-94.191716,[]
644,4993557204,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,[]
1009,3053198401,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,[]
1010,9592567002,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,[]
2210,2917617110,OK,"1715 South Highway 183, Clinton, OK",30.220229,-97.683076,[]
2393,8825047105,OK,"809 NE HWY 60 809 NE HWY 60, SEILING, OK",41.285247,-99.385728,[]
2549,5893907210,OK,"1003 HWY 64 NORTH, BUFFALO, OK",35.922512,-75.678195,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11621/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
64,4920347210,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
423,6223537005,OK,"7001 S Hwy 69A, MIAMI, OK",35.078999,-106.715022,NaN
491,3319208509,OK,"7600 S Highway 69A, Miami, OK",37.196334,-94.191716,NaN
573,8230827006,OK,"7600 S Hwy 69 A, MIAMI, OK",37.196334,-94.191716,NaN
644,4993557204,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
1009,3053198401,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
1010,9592567002,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
2210,2917617110,OK,"1715 South Highway 183, Clinton, OK",30.220229,-97.683076,NaN
2393,8825047105,OK,"809 NE HWY 60 809 NE HWY 60, SEILING, OK",41.285247,-99.385728,NaN
2549,5893907210,OK,"1003 HWY 64 NORTH, BUFFALO, OK",35.922512,-75.678195,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
64,4920347210,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
423,6223537005,OK,"7001 S Hwy 69A, MIAMI, OK",35.078999,-106.715022,NaN
491,3319208509,OK,"7600 S Highway 69A, Miami, OK",37.196334,-94.191716,NaN
573,8230827006,OK,"7600 S Hwy 69 A, MIAMI, OK",37.196334,-94.191716,NaN
644,4993557204,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
1009,3053198401,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
1010,9592567002,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
2210,2917617110,OK,"1715 South Highway 183, Clinton, OK",30.220229,-97.683076,NaN
2393,8825047105,OK,"809 NE HWY 60 809 NE HWY 60, SEILING, OK",41.285247,-99.385728,NaN
2549,5893907210,OK,"1003 HWY 64 NORTH, BUFFALO, OK",35.922512,-75.678195,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
64,4920347210,NaN
423,6223537005,NaN
491,3319208509,NaN
573,8230827006,NaN
644,4993557204,NaN
1009,3053198401,NaN
1010,9592567002,NaN
2210,2917617110,NaN
2393,8825047105,NaN
2549,5893907210,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/OK_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,710041,1008918710,OK,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14
1,710042,1994657208,OK,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,4.010911e+14
2,710043,2427157100,OK,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14
3,710044,6097937201,OK,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14
4,710045,6262678403,OK,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,4.013995e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,710041,1008918710,OK,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14,NaN
1,710042,1994657208,OK,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,4.010911e+14,NaN
2,710043,2427157100,OK,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14,NaN
3,710044,6097937201,OK,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14,NaN
4,710045,6262678403,OK,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,4.013995e+14,NaN
...,...,...,...,...,...,...,...,...
9990,720031,8894878401,OK,"1110 Cornwell Dr, Yukon, OK",35.496727,-97.742540,4.001730e+14,NaN
9991,720032,8956648300,OK,"205 S Main St, Sapulpa, OK",35.996254,-96.114040,4.003702e+14,NaN
9992,720033,9099848507,OK,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,4.013503e+14,NaN
9993,720034,9183018301,OK,"2640 NW 2nd St, Oklahoma City, OK",35.470173,-97.559910,4.010911e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,710041,1008918710,OK,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14,NaN,401310505011056.0
1,710042,1994657208,OK,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,4.010911e+14,NaN,401091066083007.0
2,710043,2427157100,OK,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14,NaN,401430059001003.0
3,710044,6097937201,OK,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14,NaN,401091093003007.0
4,710045,6262678403,OK,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,4.013995e+14,NaN,401399509021000.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1008918710,OK,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,401310505011056.0
1,1994657208,OK,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,401091066083007.0
2,2427157100,OK,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,401430059001003.0
3,6097937201,OK,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,401091093003007.0
4,6262678403,OK,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,401399509021000.0
...,...,...,...,...,...,...
9990,8894878401,OK,"1110 Cornwell Dr, Yukon, OK",35.496727,-97.742540,400173012014005.0
9991,8956648300,OK,"205 S Main St, Sapulpa, OK",35.996254,-96.114040,400370213002020.0
9992,9099848507,OK,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,401350301011054.0
9993,9183018301,OK,"2640 NW 2nd St, Oklahoma City, OK",35.470173,-97.559910,401091100003019.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
64,4920347210,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
423,6223537005,OK,"7001 S Hwy 69A, MIAMI, OK",35.078999,-106.715022,NaN
491,3319208509,OK,"7600 S Highway 69A, Miami, OK",37.196334,-94.191716,NaN
573,8230827006,OK,"7600 S Hwy 69 A, MIAMI, OK",37.196334,-94.191716,NaN
644,4993557204,OK,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
1009,3053198401,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
1010,9592567002,OK,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
2210,2917617110,OK,"1715 South Highway 183, Clinton, OK",30.220229,-97.683076,NaN
2393,8825047105,OK,"809 NE HWY 60 809 NE HWY 60, SEILING, OK",41.285247,-99.385728,NaN
2549,5893907210,OK,"1003 HWY 64 NORTH, BUFFALO, OK",35.922512,-75.678195,NaN


Convert to File

In [19]:
fips_OK = fips_merge_drop

In [20]:
fips_OK.to_csv("../../../data/state_data/geo/geo_fips/23/OK_fips_scraped.csv") 